In [1]:
import sys
sys.path.append('/home/jam1042/anaconda3/envs/skl_pys/lib/python3.9/site-packages/')
import numpy as np
from qmlearn.model import QMModel
from qmlearn.drivers.mol import QMMol
from qmlearn.preprocessing import get_train_atoms
from qmlearn.io.hdf5 import DBHDF5
from ase import Atoms
from ase.visualize import view

In [2]:
# npoints = 150
# R_begin=0.3
# R_end=4.5
# R = np.linspace(start=R_begin,stop=R_end,num=npoints)
HARTREE_TO_eV = 27.311285

'''Data set _b:Bigger Basis cc-ptvz, _s: small basis, base_name: small stepsize'''

'Data set _b:Bigger Basis cc-ptvz, _s: small basis, base_name: small stepsize'

In [3]:
npoints_1=50
R1_begin=0.05
R1_end=0.7
R1=np.linspace(start=R1_begin,stop=R1_end,num=npoints_1)

npoints_2 = 130
R2_begin=0.7
R2_end=4.5
R2 = np.linspace(start=R2_begin,stop=R2_end,num=npoints_2)

R=np.concatenate([R1,R2])
npoints=npoints_1+npoints_2
# np.shape(R)

In [4]:
HARTREE_TO_eV = 27.311285

In [5]:
split=int(npoints*0.9)
split

162

In [6]:
# Geometries to work with for LiH
mol_LiH="Li 0 0 0; H 0 0 "
mol_stra_LiH = []
for i in np.arange(npoints):
    mol_stra_LiH.append(str(mol_LiH)+str(R[i]))

In [7]:
#Keep all the geometries in ASE format
liH_atoms=[]
for i in np.arange(npoints):
    liH_atoms.append(Atoms('LiH',positions=[(0,0,0),(0,0,R[i])]))
# liH_atoms    

In [8]:
# for i in np.arange(0,len(liH_atoms)):
#     print(i)
#     print(liH_atoms[i].get_positions())

In [9]:
# Properties to be calculated
prop = ['vext', 'gamma','gamma2', 'energy', 'ke', 'dipole']
# prop = ['vext', 'gamma', 'energy', 'ke', 'dipole']
properties= { k: [] for k in prop}
test_data = { k: [] for k in prop}

#### Creating Data Based for LiH

In [10]:
# Defining the Training SET
train_atoms=liH_atoms[0:split]  # Zero to Five Geometries for LiH
nsamples = len(train_atoms)
nsamples

162

In [11]:
#Keep a Ref qmmol
basis='6-31g*'
method = 'fci'
# method = 'hf'
refqmmol = QMMol(atoms = liH_atoms[0],  method = method, basis=basis)

In [12]:
def gamma2props(qmmol, gamma,gamma2, etotal, p, prop=[]):
# def gamma2props(qmmol, gamma,etotal, p, prop=[]):
    if 'vext' in prop:
        p['vext'].append(qmmol.engine.vext)
    if 'gamma' in prop:
        p['gamma'].append(gamma)
    if 'gamma2' in prop:
        p['gamma2'].append(gamma2)
    if 'energy' in prop:
        p['energy'].append(etotal)
    if 'ke' in prop:
        p['ke'].append(qmmol.calc_ke(gamma))
    if 'dipole' in prop:
        p['dipole'].append(qmmol.calc_dipole(gamma))
    return p

In [13]:
# Obtaining properties or feature for Train_Atoms

for atoms in train_atoms:
    qmmol = refqmmol.duplicate(atoms)
    qmmol.run()
    pp=properties
    gamma=qmmol.engine.gamma
    gamma2=qmmol.engine.gammat
    etotal=qmmol.engine.etotal
    gamma2props(qmmol, gamma, gamma2, etotal, pp, prop=prop[:-1])
#     gamma2props(qmmol, gamma, etotal, pp, prop=prop[:-1])
    pp['energy'][-1] = qmmol.engine.etotal

    #change gamma2 to 2gamma

In [14]:
test_atoms = liH_atoms[split:]
nsamples = len(test_atoms)
nsamples

18

In [15]:
for atoms in test_atoms:
    qmmol = refqmmol.duplicate(atoms)
    qmmol.run()
    p = test_data
    gamma = qmmol.engine.gamma
    gamma2=qmmol.engine.gammat
    etotal=qmmol.engine.etotal
    gamma2props(qmmol, gamma, gamma2, etotal, p, prop=prop[:-1])
#     gamma2props(qmmol, gamma, etotal, p, prop=prop[:-1])
    p['energy'][-1] = qmmol.engine.etotal

In [16]:
dbfile = 'LiH_QML_set_f.hdf5'
db = DBHDF5(dbfile, qmmol=refqmmol)

In [17]:
input_options={
    'basis': basis,
    'method': method,
}

In [18]:
properties['input_options']=input_options
db.write_qmmol(refqmmol)
db.write_images(train_atoms, prefix='train')
db.write_properties(properties, prefix='train')
db.write_images(test_atoms, prefix='test')
db.write_properties(test_data, prefix='test')
db.names

['fci',
 'fci/qmmol',
 'fci/test_atoms_18',
 'fci/test_props_18',
 'fci/train_atoms_162',
 'fci/train_props_162']

In [19]:
db.close()

In [20]:
db = DBHDF5(dbfile, qmmol=refqmmol)
data = db.read_properties(db.get_names('*/train_props_*')[0])
db.close()

data['energy'][:3]

array([17.73807709, 11.27536108,  7.11802818])

#### Creating Data Based for H2

In [21]:
# Geometries to work with for H2 
mol_h2="H 0 0 0; H 0 0 "
mol_stra_h2 = []
for i in np.arange(npoints):
    mol_stra_h2.append(str(mol_h2)+str(R[i]))

In [22]:
H2_atoms=[]
for i in np.arange(npoints):
    H2_atoms.append(Atoms('HH',positions=[(0,0,0),(0,0,R[i])]))

In [23]:
prop = ['vext', 'gamma','gamma2', 'energy', 'ke', 'dipole']

properties= { k: [] for k in prop}
test_data = { k: [] for k in prop}

In [24]:
# Defining the Training SET
train_atoms=H2_atoms[0:split]  # Zero to Five Geometries for H2
nsamples = len(train_atoms)
nsamples

162

In [25]:
#Keep a Ref qmmol
basis='6-31g*'
method = 'fci'
# method= 'hf'
refqmmol = QMMol(atoms = H2_atoms[0],  method = method, basis=basis)

In [26]:
def gamma2props(qmmol, gamma,gamma2, etotal, p, prop=[]):
# def gamma2props(qmmol, gamma, etotal, p, prop=[]):
    if 'vext' in prop:
        p['vext'].append(qmmol.engine.vext)
    if 'gamma' in prop:
        p['gamma'].append(gamma)
    if 'gamma2' in prop:
        p['gamma2'].append(gamma2)
    if 'energy' in prop:
        p['energy'].append(etotal)
    if 'ke' in prop:
        p['ke'].append(qmmol.calc_ke(gamma))
    if 'dipole' in prop:
        p['dipole'].append(qmmol.calc_dipole(gamma))
    return p

In [27]:
# Obtaining properties or feature for Train_Atoms

for atoms in train_atoms:
    qmmol = refqmmol.duplicate(atoms)
    qmmol.run()
    pp=properties
    gamma=qmmol.engine.gamma
    gamma2=qmmol.engine.gammat
    etotal=qmmol.engine.etotal
    gamma2props(qmmol, gamma, gamma2, etotal, pp, prop=prop[:-1])
#     gamma2props(qmmol, gamma, etotal, pp, prop=prop[:-1])
    pp['energy'][-1] = qmmol.engine.etotal

In [28]:
test_atoms = H2_atoms[split:]
nsamples = len(test_atoms)
nsamples

18

In [29]:
for atoms in test_atoms:
    qmmol = refqmmol.duplicate(atoms)
    qmmol.run()
    p = test_data
    gamma = qmmol.engine.gamma
    gamma2=qmmol.engine.gammat
    etotal=qmmol.engine.etotal
    gamma2props(qmmol, gamma, gamma2, etotal, p, prop=prop[:-1])
#     gamma2props(qmmol, gamma, etotal, p, prop=prop[:-1])
    p['energy'][-1] = qmmol.engine.etotal

In [30]:
dbfile = 'H2_QML_set_f.hdf5'
db = DBHDF5(dbfile, qmmol=refqmmol)

In [31]:
input_options={
    'basis': basis,
    'method': method,
}

In [32]:
properties['input_options']=input_options
db.write_qmmol(refqmmol)
db.write_images(train_atoms, prefix='train')
db.write_properties(properties, prefix='train')
db.write_images(test_atoms, prefix='test')
db.write_properties(test_data, prefix='test')
db.names

['fci',
 'fci/qmmol',
 'fci/test_atoms_18',
 'fci/test_props_18',
 'fci/train_atoms_162',
 'fci/train_props_162']

In [33]:
db.close()

In [34]:
db = DBHDF5(dbfile, qmmol=refqmmol)
data = db.read_properties(db.get_names('*/train_props_*')[0])
db.close()

len(data)

7

In [35]:
data

{'dipole': array([], dtype=float64),
 'energy': array([ 7.8153786 ,  5.60640499,  4.16863264,  3.16088494,  2.41751909,
         1.84835325,  1.40005907,  1.03906483,  0.74317336,  0.49711657,
         0.29004495,  0.11403513, -0.03683611, -0.1670873 , -0.28022701,
        -0.37902257, -0.465687  , -0.54201183, -0.60946315, -0.66925226,
        -0.72238848, -0.76971921, -0.81196077, -0.84972245, -0.88352547,
        -0.91381807, -0.94098769, -0.96537075, -0.9872607 , -1.00691457,
        -1.02455831, -1.04039132, -1.05459013, -1.06731148, -1.07869497,
        -1.0888652 , -1.0979337 , -1.10600048, -1.11315544, -1.11947953,
        -1.1250458 , -1.12992027, -1.13416274, -1.13782744, -1.14096365,
        -1.14361624, -1.1458261 , -1.14763054, -1.1490637 , -1.15015683,
        -1.15015683, -1.15150881, -1.1515958 , -1.15064562, -1.14884823,
        -1.14636245, -1.14332143, -1.13983691, -1.13600285, -1.13189827,
        -1.12758969, -1.12313316, -1.11857593, -1.11395786, -1.10931259,
    